## Imports

In [4]:
import sys
sys.path.append("C:/Users/AndrejTschalzev/Desktop/InES/Mixed_Effects/menn/model")
# sys.path.append("C:/Users/AndrejTschalzev/Desktop/InES/Mixed_Effects/lmmnn-main/lmmnn/")
sys.path.append("C:/Users/AndrejTschalzev/Desktop/InES/Mixed_Effects/Simchoni Repo/lmmnn/lmmnn")
import os
aPath = '--xla_gpu_cuda_data_dir=C:/Users/AndrejTschalzev/anaconda3/Library/bin'
print(aPath)
os.environ['XLA_FLAGS'] = aPath

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from MENBernoulli import MEFFNN_Bernoulli
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score as acc
from tensorflow.keras.activations import sigmoid
from keras.models import Sequential, Model
from keras.layers import Dense,LSTM,GRU,Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam

# TODO: Change following import!!!
#from utils import generate_data
from utils import generate_data 
from layers import NLL
from nn import reg_nn_lmm

--xla_gpu_cuda_data_dir=C:/Users/AndrejTschalzev/anaconda3/Library/bin


## Load/Generate Data

In [5]:
params = {
    "n_fixed_effects": 10,
    "fixed_intercept": 1,
    "X_non_linear": True,
    "Z_non_linear": False,
    "Z_embed_dim_pct": 10,
    "n_per_cat": 3,
    "test_size": 0.2,
}


N = 100000
d = params["n_fixed_effects"]
q = 1000
sig2b = 1.0
mode = "glmm"

X_train, X_test, y_train, y_test, x_cols, dist_matrix, time2measure_dict = generate_data(mode=mode, qs=[q], sig2e=1, sig2bs=[sig2b], sig2bs_spatial=[], q_spatial=[], N=N, rhos=None, p_censor=None, params=params)

In [6]:
# set dataframes for LMMNN of Simchoni
X_train_lmmnn = X_train.copy()
X_test_lmmnn = X_test.copy()
y_train_lmmnn = y_train.copy()
y_test_lmmnn = y_test.copy()

In [7]:
# define dataframes for other models
Z_train = X_train["z0"].values
X_train = X_train[x_cols].astype(np.float32)
y_train = y_train.astype(np.float32)
Z_test = X_test["z0"].values
X_test = X_test[x_cols].astype(np.float32)
y_test = y_test.astype(np.float32)

In [8]:
# set dataframes for MENN 
X_train_menn = X_train.copy()
X_test_menn = X_test.copy()
y_train_menn = y_train.copy()
y_test_menn = y_test.copy()
Z_train_menn = Z_train.copy()
Z_test_menn = Z_test.copy()

# set dataframes for Target Encoding Model
X_train_te = X_train.copy()
X_test_te = X_test.copy()
y_train_te = y_train.copy()
y_test_te = y_test.copy()
Z_train_te = Z_train.copy()
Z_test_te = Z_test.copy()

## Modelling Comparison

### 1) Standard Neural Network without Random Effects Variable

In [9]:
# simple model with 1 hidden layer
model_simple = Sequential()
model_simple.add(Dense(100, activation='relu'))
model_simple.add(Dense(50, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train, y_train, epochs=20, batch_size=100)
# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
800/800 [==============================] - 3s 2ms/step - loss: 0.9229 - accuracy: 0.5972
Epoch 2/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6301 - accuracy: 0.6387
Epoch 3/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6237 - accuracy: 0.6443
Epoch 4/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6164 - accuracy: 0.6513
Epoch 5/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6093 - accuracy: 0.6575
Epoch 6/20
800/800 [==============================] - 1s 2ms/step - loss: 0.6026 - accuracy: 0.6620
Epoch 7/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5971 - accuracy: 0.6672
Epoch 8/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5915 - accuracy: 0.6733
Epoch 9/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5873 - accuracy: 0.6779
Epoch 10/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5840 - accuracy: 0.6804

### 2) LMMNN of Simchoni

In [10]:
x_cols = [col for col in X_train.columns if col not in ['z0','z1','z2']]
batch = 100
epochs = 20
patience = 10
qs=[q]
q_spatial=[]
n_neurons = [100,50]
dropout = None
activation = 'relu'
mode = 'glmm'
n_sig2bs = 1
n_sig2bs_spatial = 0
est_cors = []
dist_matrix = None
time2measure_dict = None
spatial_embed_neurons = None
verbose = True
Z_non_linear = False
log_params = False
idx = None
Z_embed_dim_pct = 10

In [11]:
y_pred, sigmas, rhos, weibull, n_epochs = reg_nn_lmm(
            X_train_lmmnn, X_test_lmmnn, y_train_lmmnn, y_test_lmmnn, qs, q_spatial, x_cols, batch, epochs, patience,
            n_neurons, dropout, activation, mode,
            n_sig2bs, n_sig2bs_spatial, est_cors, dist_matrix, spatial_embed_neurons, verbose, Z_non_linear, Z_embed_dim_pct, log_params, idx)

Epoch 1/20
720/720 [==============================] - 11s 12ms/step - loss: 58.5966 - val_loss: 55.3959
Epoch 2/20
720/720 [==============================] - 8s 12ms/step - loss: 53.6160 - val_loss: 52.1782
Epoch 3/20
720/720 [==============================] - 9s 12ms/step - loss: 51.8880 - val_loss: 51.4472
Epoch 4/20
720/720 [==============================] - 8s 12ms/step - loss: 51.4443 - val_loss: 51.2510
Epoch 5/20
720/720 [==============================] - 8s 11ms/step - loss: 51.2681 - val_loss: 51.1577
Epoch 6/20
720/720 [==============================] - 9s 12ms/step - loss: 51.1657 - val_loss: 51.1195
Epoch 7/20
720/720 [==============================] - 9s 12ms/step - loss: 51.1007 - val_loss: 51.0864
Epoch 8/20
720/720 [==============================] - 8s 11ms/step - loss: 51.0516 - val_loss: 51.0792
Epoch 9/20
720/720 [==============================] - 8s 11ms/step - loss: 51.0121 - val_loss: 51.0721
Epoch 10/20
720/720 [==============================] - 9s 12ms/step - lo

In [12]:
# round predictions to closest integer and compute accuracy
y_pred_class = []
for n in y_pred:
    y_pred_class.append(round(n))

acc(y_test_lmmnn, y_pred_class)

0.74845

### 3) MENN of Andrej

In [26]:
optimizer = Adam(learning_rate=.0001)#, decay=0.0001)

model_jointdist = MEFFNN_Bernoulli(d,q, layers=[100,50])
model_jointdist.fit(X_train_menn, y_train_menn, Z_train_menn, optimizer=optimizer, num_warmup_iters=1000, num_iters=5000, print_warmup=1500, print_training=500)

Warm-Up Iteration:   0 Acceptance Rate: 1.000
Warm-Up Iteration: 999 Acceptance Rate: 1.000
Iteration:    0 Acceptance Rate: 1.000 Loss: 53246.059
Iteration:  500 Acceptance Rate: 0.994 Loss: 46992.707
Iteration: 1000 Acceptance Rate: 0.995 Loss: 44494.578
Iteration: 1500 Acceptance Rate: 0.996 Loss: 42192.867
Iteration: 2000 Acceptance Rate: 0.997 Loss: 40920.500
Iteration: 2500 Acceptance Rate: 0.997 Loss: 40323.957
Iteration: 3000 Acceptance Rate: 0.997 Loss: 39978.156
Iteration: 3500 Acceptance Rate: 0.997 Loss: 39875.992
Iteration: 4000 Acceptance Rate: 0.997 Loss: 39750.750
Iteration: 4500 Acceptance Rate: 0.996 Loss: 39747.520
Iteration: 4999 Acceptance Rate: 0.996 Loss: 39581.137


In [27]:
y_train_pred = model_jointdist.predict(X_train_menn,Z_train_menn)
y_train_pred_fe = model_jointdist.predict(X_train_menn)
y_test_pred = model_jointdist.predict(X_test_menn,Z_test_menn)
y_test_pred_fe = model_jointdist.predict(X_test_menn)

In [28]:
## Evaluation on Train
print("Accuracy: ", acc(y_train_menn, np.round(y_train_pred)))
print("Accuracy (only FE): ", acc(y_train_menn, np.round(y_train_pred_fe)))

mean_y_group = pd.DataFrame([Z_train_menn,y_train_menn]).transpose().groupby(0).mean().values.ravel()
print("Correlation mean occupation/raneffs: ", np.corrcoef([model_jointdist.random_effects, mean_y_group])[0,1])

print("Meaningful RE indicator value: ", (np.corrcoef([model_jointdist.random_effects, mean_y_group])[0,1]-np.corrcoef([np.random.randn(model_jointdist.random_effects.shape[0]), model_jointdist.random_effects])[0,1]).round(3))

print("STd of RE's: ", model_jointdist._stddev_z0.numpy())

Accuracy:  0.769125
Accuracy (only FE):  0.7036125
Correlation mean occupation/raneffs:  0.9843849924029732
Meaningful RE indicator value:  0.95
STd of RE's:  1.0236143


In [29]:
## Evaluation on Test
print("Accuracy: ", acc(y_test_menn, np.round(y_test_pred)))
print("Accuracy (only FE): ", acc(y_test_menn, np.round(y_test_pred_fe)))

mean_y_group = pd.DataFrame([Z_test_menn,y_test_menn]).transpose().groupby(0).mean().values.ravel()
print("Correlation mean occupation/raneffs: ", np.corrcoef([model_jointdist.random_effects, mean_y_group])[0,1])

### Simple check if RE's correlation to group means is not random - values close to 1 indicate high meaningfulness, while close to zero indicates randomness
print("Meaningful RE indicator value: ", (np.corrcoef([model_jointdist.random_effects, mean_y_group])[0,1]-np.corrcoef([np.random.randn(model_jointdist.random_effects.shape[0]), model_jointdist.random_effects])[0,1]).round(3))

print("STd of RE's: ", model_jointdist._stddev_z0.numpy())

Accuracy:  0.74735
Accuracy (only FE):  0.68905
Correlation mean occupation/raneffs:  0.7704344211182708
Meaningful RE indicator value:  0.736
STd of RE's:  1.0236143


### 4) Target Encoding

In [19]:
from category_encoders import TargetEncoder

In [20]:
X_train_te['z0'] = Z_train_te
X_test_te['z0'] = Z_test_te
X_train_te['target'] = y_train_te
X_test_te['target'] = y_test_te

df_train = X_train_te
df_test = X_test_te

In [21]:
# transform random effect variable to string
df_train.z0 = df_train.z0.astype(str)
df_test.z0 = df_test.z0.astype(str)

In [22]:
encoder = TargetEncoder()
df_train['RE Encoded'] = encoder.fit_transform(df_train['z0'], df_train['target']) # fit_transform on train data
df_test['RE Encoded'] = encoder.transform(df_test['z0'], df_test['target']) # transform on test data
df_train

C:\Users\AndrejTschalzev\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\AndrejTschalzev\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,z0,target,RE Encoded
67289,-0.988776,0.892862,0.069132,-0.872051,-0.410435,-0.492521,-0.602586,0.549388,-0.492385,-0.583401,674,0.0,0.278049
61772,-0.527640,-0.966804,-0.178469,0.814343,0.857220,0.208040,-0.114703,0.417908,-0.955980,0.770787,619,1.0,0.481481
58427,-0.798362,-0.215860,0.610483,-0.600022,0.181420,-0.009999,0.352461,0.090399,0.702602,0.983926,586,1.0,0.670886
75760,0.057209,0.740510,-0.216223,0.800969,0.242166,0.734752,0.370333,-0.963477,-0.010866,0.126177,750,0.0,0.278481
52771,0.385098,0.456060,-0.201385,-0.345627,0.250710,0.362323,-0.272217,-0.588831,-0.653261,-0.972687,531,1.0,0.670455
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34953,-0.787268,-0.243127,0.792764,0.208276,-0.506343,0.207648,0.236252,0.470231,0.817081,-0.762650,358,1.0,0.317829
51333,0.031326,0.037340,-0.229243,0.498082,0.336013,0.225062,0.325180,-0.405215,0.890586,-0.328829,520,0.0,0.654321
46890,0.816116,0.270360,0.554139,0.784908,-0.004384,-0.219889,-0.272515,0.600443,0.201243,-0.737825,474,0.0,0.243386
37331,0.164079,0.286051,-0.201195,-0.413490,-0.428126,-0.380729,0.098788,0.037482,0.161009,0.761972,379,1.0,0.404255


In [23]:
# drop z0
df_train = df_train.drop(['z0'], axis=1)
df_test = df_test.drop(['z0'], axis=1)

In [24]:
# define y and X variables
y_train_te = df_train['target']
X_train_te = df_train.drop(['target'], axis=1)
y_test_te = df_test['target']
X_test_te = df_test.drop(['target'], axis=1)

In [25]:
# simple model with 1 hidden layer
model_simple = Sequential()
model_simple.add(Dense(100, activation='relu'))
model_simple.add(Dense(50, activation='sigmoid'))
model_simple.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    
model_simple.fit(X_train_te, y_train_te, epochs=20, batch_size=100)
# Final evaluation of the model
scores = model_simple.evaluate(X_test_te, y_test_te, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 0.8353 - accuracy: 0.6427
Epoch 2/20
800/800 [==============================] - 2s 2ms/step - loss: 0.5606 - accuracy: 0.7131
Epoch 3/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7190
Epoch 4/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5427 - accuracy: 0.7244
Epoch 5/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5344 - accuracy: 0.7306
Epoch 6/20
800/800 [==============================] - 2s 2ms/step - loss: 0.5276 - accuracy: 0.7355
Epoch 7/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5209 - accuracy: 0.7402
Epoch 8/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5153 - accuracy: 0.7454
Epoch 9/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5109 - accuracy: 0.7480
Epoch 10/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5072 - accuracy: 0.7518